---
# GPU Memory Stress Test
---

In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
import torch
import time

In [2]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("✅ CUDA is available!")
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    print("❌ CUDA is NOT available. Using CPU.")
    device = torch.device("cpu")

✅ CUDA is available!
Using GPU: NVIDIA GeForce GTX 1650
